## 다중 선형 회귀(Multivariable Linear Regression)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

독립변수 3개의 x 로부터 종속변수 y 예측
>H(x) = w1x1 + w2x2 + w3x3 + b

In [3]:
x1_train = torch.FloatTensor([[73],[93],[89],[96],[73]])
x2_train = torch.FloatTensor([[80],[88],[91],[98],[66]])
x3_train = torch.FloatTensor([[75],[93],[90],[100],[70]])
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])

In [4]:
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [9]:
# 가설, 비용함수, 옵티마이저를 선언한 후에 경사하강법을 1000회 반복
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs=1000
for epoch in range(nb_epochs+1):
    hypothesis = x1_train*w1 + x2_train*w2 + x3_train*w3 + b
    cost = torch.mean((hypothesis - y_train)**2)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 ==0:
        print('epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3 : {:.3f} b : {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item())) 

epoch    0/1000 w1: 0.757 w2: 0.571 w3 : 0.682 b : 0.011 Cost: 0.753893
epoch  100/1000 w1: 0.761 w2: 0.568 w3 : 0.682 b : 0.011 Cost: 0.729910
epoch  200/1000 w1: 0.764 w2: 0.564 w3 : 0.682 b : 0.011 Cost: 0.707164
epoch  300/1000 w1: 0.767 w2: 0.561 w3 : 0.682 b : 0.011 Cost: 0.685575
epoch  400/1000 w1: 0.770 w2: 0.558 w3 : 0.682 b : 0.011 Cost: 0.665089
epoch  500/1000 w1: 0.773 w2: 0.555 w3 : 0.682 b : 0.011 Cost: 0.645660
epoch  600/1000 w1: 0.776 w2: 0.552 w3 : 0.682 b : 0.011 Cost: 0.627213
epoch  700/1000 w1: 0.779 w2: 0.549 w3 : 0.682 b : 0.012 Cost: 0.609713
epoch  800/1000 w1: 0.782 w2: 0.546 w3 : 0.682 b : 0.012 Cost: 0.593101
epoch  900/1000 w1: 0.785 w2: 0.543 w3 : 0.682 b : 0.012 Cost: 0.577335
epoch 1000/1000 w1: 0.788 w2: 0.540 w3 : 0.682 b : 0.012 Cost: 0.562360


### 벡터와 행렬 연산으로 바꾸기
> Dot Product : H(X) = XW + B

In [12]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

x_trian 하나에 모든 샘플을 전부 선언 (5*3) 행렬 X선언.
- 전체 훈련 데이터의 개수를 셀 수 있는 1개의 단위를 sample이라고 하고 현재 5개.
- 각 샘플에서 y를 결정하게 하는 각각의 독립변수 x를 feature라고 함. 현재 3개
이는 종속변수 x들의 수가 15개 임을 의미하며 종속변수 x들을 (샘플의 수*특성의 수)의 크기를 가지는 하나의 행렬로 표현 가능


> 결과적으로 가중치 (w1, w2, w3)행렬 W와 종속변수 행렬 X의 곱(5*3), bias(b)행렬의 덧셈으로 식변경이 가능

> H(x) = w1x1 + w2x2 + w3x3 + b ~ H(X) = XW+b

In [17]:
print(x_train.shape)
print(y_train.shape)


torch.Size([5, 3])
torch.Size([5, 1])


In [14]:
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [16]:
optimizer = optim.SGD([W, b], lr=1e-5)
nb_epochs = 20
for epoch in range(nb_epochs+1):
    hypothesis = x_train.matmul(W)+b
    cost = torch.mean((hypothesis - y_train)**2)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712646
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371017
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo